In [14]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
import pyLDAvis
from pyLDAvis import sklearn as sklearn_lda
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from itertools import chain 
from langdetect import detect
from gensim.models import Phrases
from gensim import corpora
from gensim import models
import pyLDAvis
import pyLDAvis.gensim

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

tqdm.pandas()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
#set the parameters
stopwords_other=['middle','aged','author','intervention','followup','improve','conclusion',
                 'background','patient','group','cancer','patients','groups','study','treatment',
                'control','significant','effects','effect','trial','interventions', 'year','respectively','data'
                 'exercice','exercices','compare','clinical','breast','breasts','include', 'compare', 'recieve', 'use', 'group', 'response','intervention']
NUM_TOPICS = 5

In [0]:
df_yes = pd.read_csv("yeah.csv", sep = ',')
df_no = pd.read_csv("no_not_clean_from_json.csv", sep = ',')
df_text_yes=df_yes.loc[:,['text', 'response','pmid']]
df_text_yes['response'].replace({1: 0}, inplace=True) 
df_text_no=df_no.loc[:,['text', 'response','pmid']]
df_text_no['response'].replace({2: 1}, inplace=True) 

#create a list of all pmid. We keep only lengthX_yes valus for No INM
df1 = pd.DataFrame(df_text_yes['pmid'],columns=['pmid'])
df2 = pd.DataFrame(df_text_no['pmid'][0:df_text_yes.shape[0]],columns=['pmid'])
data=pd.concat([df_text_yes,df_text_no[0:df_text_yes.shape[0]]])
df_pmids=pd.concat([df1,df2])

#reindex the dataset in order to have the same numbers to both
df_pmids.reset_index(drop=True, inplace=True)

data.reset_index(drop=True, inplace=True)
data['text'] = data['text'].astype(str)

In [17]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

data['sentences'] = data['text'].progress_map(sent_tokenize)
data['tokens_sentences'] = data['sentences'].progress_map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
data['POS_tokens'] = data['tokens_sentences'].progress_map(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])

    
lemmatizer = WordNetLemmatizer()

Streaming output truncated to the last 5000 lines.
 58%|█████▊    | 32796/56600 [05:36<05:13, 75.82it/s]

 58%|█████▊    | 32804/56600 [05:36<05:10, 76.58it/s]

 58%|█████▊    | 32814/56600 [05:36<04:56, 80.31it/s]

 58%|█████▊    | 32823/56600 [05:36<04:50, 81.90it/s]

 58%|█████▊    | 32833/56600 [05:36<04:38, 85.48it/s]

 58%|█████▊    | 32842/56600 [05:37<04:36, 85.84it/s]

 58%|█████▊    | 32851/56600 [05:37<04:37, 85.57it/s]

 58%|█████▊    | 32860/56600 [05:37<04:42, 83.89it/s]

 58%|█████▊    | 32869/56600 [05:37<04:54, 80.50it/s]

 58%|█████▊    | 32878/56600 [05:37<05:08, 76.97it/s]

 58%|█████▊    | 32887/56600 [05:37<05:00, 78.80it/s]

 58%|█████▊    | 32897/56600 [05:37<04:43, 83.58it/s]

 58%|█████▊    | 32907/56600 [05:37<04:34, 86.16it/s]

 58%|█████▊    | 32916/56600 [05:37<04:33, 86.65it/s]

 58%|█████▊    | 32925/56600 [05:38<04:47, 82.22it/s]

 58%|█████▊    | 32935/56600 [05:38<04:47, 82.31it/s]

 58%|█████▊    | 32944/56600 [05:38<04:46, 82.65it/s]

 58%|█████▊   

In [21]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
data['tokens_sentences_lemmatized'] = data['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

my_stopwords = stopwords.words('english') + stopwords_other
data['tokens'] = data['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
data['tokens'] = data['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>1])

tokens = data['tokens'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]




  0%|          | 0/56600 [00:00<?, ?it/s]


  0%|          | 92/56600 [00:00<01:01, 919.33it/s]


  0%|          | 188/56600 [00:00<01:00, 930.99it/s]


  0%|          | 279/56600 [00:00<01:00, 923.80it/s]


  1%|          | 376/56600 [00:00<01:00, 934.69it/s]


  1%|          | 468/56600 [00:00<01:00, 929.34it/s]


  1%|          | 549/56600 [00:00<01:03, 884.08it/s]


  1%|          | 647/56600 [00:00<01:01, 910.77it/s]


  1%|▏         | 787/56600 [00:00<00:54, 1017.06it/s]


  2%|▏         | 905/56600 [00:00<00:52, 1059.68it/s]


  2%|▏         | 1031/56600 [00:01<00:49, 1111.84it/s]


  2%|▏         | 1143/56600 [00:01<00:49, 1112.13it/s]


  2%|▏         | 1255/56600 [00:01<00:50, 1086.09it/s]


  2%|▏         | 1379/56600 [00:01<00:48, 1127.43it/s]


  3%|▎         | 1501/56600 [00:01<00:47, 1152.26it/s]


  3%|▎         | 1631/56600 [00:01<00:46, 1191.79it/s]


  3%|▎         | 1757/56600 [00:01<00:45, 1209.55it/s]


  3%|▎         | 1879/56600 [00:01<00:45, 1203.76it/s]


 

In [24]:
np.random.seed(123456)

lda_model = models.LdaModel(corpus, num_topics=NUM_TOPICS, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*NUM_TOPICS, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

In [0]:
def topics_document_to_dataframe(topics_document, num_topics):
    res = pd.DataFrame(columns=range(num_topics))
    for topic_weight in topics_document:
        res.loc[0, topic_weight[0]] = topic_weight[1]
    return res

topics = [lda_model[corpus[i]] for i in range(len(data))]

document_topic = \
pd.concat([topics_document_to_dataframe(topics_document, num_topics=NUM_TOPICS) for topics_document in topics]) \
  .reset_index(drop=True).fillna(0)


docs=document_topic.copy()
docs ['pmid']=data['pmid'].values
docs['response']=data['response'].values

def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['topic_dominant', 'pourcentage_contrib', 'topic_keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


In [26]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data['text'])
final_data=data.copy()
final_data = df_topic_sents_keywords.reset_index()
final_data.columns = ['pmid','topic_dominant', 'pourcentage_contrib', 'topic_keywords', 'text']
final_data = final_data.assign(pmid=df_pmids['pmid'])
final_data['response']=data['response'].values

print ("Les topics : ")
for i,topic in lda_model.show_topics(formatted=True, num_topics=NUM_TOPICS, num_words=20):
    print(str(i)+": "+ topic)
    print()
print ("La répartition : ")    
print (final_data.groupby(['topic_dominant','response']).size())

for i in range(NUM_TOPICS):
    new_df = final_data[(final_data['topic_dominant']==i) & (final_data['response']==0)]
    print ("valeur moyenne, min, max pour le topic ",i, " des INM ",
           new_df.pourcentage_contrib.mean(),new_df.pourcentage_contrib.min(),
          new_df.pourcentage_contrib.max())
    new_df = final_data[(final_data['topic_dominant']==i) & (final_data['response']==1)]
    print ("valeur moyenne, min, max pour le topic ",i," des non INM ",
           new_df.pourcentage_contrib.mean(),
          new_df.pourcentage_contrib.min(),
          new_df.pourcentage_contrib.max())

# Decocher les commentaires ci-dessous pour voir la répartition des clusters dans l'espace

%matplotlib inline

#vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
#pyLDAvis.enable_notebook()
#pyLDAvis.display(vis)

Les topics : 
0: 0.019*"cell" + 0.012*"increase" + 0.011*"level" + 0.009*"show" + 0.009*"may" + 0.009*"also" + 0.009*"tumor" + 0.006*"concentration" + 0.006*"human" + 0.006*"expression" + 0.005*"drug" + 0.005*"disease" + 0.005*"activity" + 0.005*"suggest" + 0.005*"high" + 0.005*"however" + 0.005*"result" + 0.004*"find" + 0.004*"serum" + 0.004*"induce"

1: 0.021*"tumor" + 0.018*"case" + 0.011*"disease" + 0.010*"survival" + 0.010*"result" + 0.009*"treat" + 0.009*"surgery" + 0.007*"perform" + 0.007*"therapy" + 0.007*"lesion" + 0.006*"carcinoma" + 0.006*"recurrence" + 0.006*"one" + 0.006*"stage" + 0.005*"show" + 0.005*"two" + 0.005*"rate" + 0.005*"follow" + 0.005*"evaluate" + 0.005*"local"

2: 0.008*"quality_life" + 0.008*"need" + 0.008*"pain" + 0.007*"report" + 0.006*"outcome" + 0.006*"information" + 0.006*"experience" + 0.006*"provide" + 0.006*"research" + 0.006*"care" + 0.005*"review" + 0.005*"may" + 0.005*"data" + 0.005*"support" + 0.005*"symptom" + 0.004*"identify" + 0.004*"also" + 0.

In [27]:
%matplotlib inline

vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)